In [ ]:
from surianalytics.connectors import RESTSciriusConnector

In [ ]:
from surianalytics.widgets.config import update_connector_timeframe

In [ ]:
CONNECTOR = RESTSciriusConnector()

In [ ]:
CONNECTOR.page_size = 10000

In [ ]:
update_connector_timeframe(c=CONNECTOR, dump_path="../config.json")

In [ ]:
from IPython.display import JSON

In [ ]:
QUERY = "event_type: fileinfo AND fileinfo.mimetype: *shell*"

In [ ]:
DATA_SCRIPTS = (
    CONNECTOR
    .get_eve_unique_values(qfilter=QUERY, 
                           field="http.url", 
                           counts="no")
)

In [ ]:
len(DATA_SCRIPTS)

In [ ]:
DATA_FLOWS = (
    CONNECTOR
    .get_eve_unique_values(qfilter=QUERY, 
                           field="flow_id", 
                           counts="no")
)

In [ ]:
len(DATA_FLOWS)

In [ ]:
JSON(CONNECTOR.get_eve_fields_graph(qfilter=QUERY, 
                                    col_src="http.url", 
                                    col_dest="flow_id"))

In [ ]:
g = CONNECTOR.get_eve_fields_graph_nx(qfilter=QUERY, col_src="src_ip", col_dest="dest_ip")

In [ ]:
from surianalytics.viz import draw_nx_graph

In [ ]:
display(draw_nx_graph(g))

In [ ]:
QUERY_TPL = "event_type: alert AND flow_id: ({flows})"

In [ ]:
BATCHSIZE = 10

In [ ]:
BATCHES = int(len(DATA_FLOWS) / BATCHSIZE) + 1

In [ ]:
DATA = []
for i in range(BATCHES):
    if len(DATA_FLOWS) > BATCHSIZE:
        batch = DATA_FLOWS[:BATCHSIZE]
        DATA_FLOWS = DATA_FLOWS[BATCHSIZE:]
    else:
        batch = DATA_FLOWS
    
    q = QUERY_TPL.format(flows=" OR ".join([str(f) for f in batch]))
    
    resp = CONNECTOR.get_events_tail(qfilter=q)
    
    DATA.extend(resp)

In [ ]:
import pandas as pd

In [ ]:
DF = pd.json_normalize(DATA)

In [ ]:
PAYLOADS = DF.payload_printable.unique()

In [ ]:
PAYLOADS

In [ ]:
!mkdir -p scripts-alerts-payloads

In [ ]:
import hashlib

In [ ]:
import os

In [ ]:
for resp in PAYLOADS:
    if not isinstance(resp, str) or len(resp) < 100:
        continue
    h = hashlib.new("sha256")
    h.update(resp.encode("utf-8"))
    with open(os.path.join("scripts-alerts-payloads", h.hexdigest()), "w") as handle:
        handle.write(resp)

In [ ]:
RESPONSES = list(DF["http.http_response_body_printable"].unique())

In [ ]:
len(RESPONSES)

In [ ]:
!mkdir -p scripts-responses

In [ ]:
for resp in RESPONSES:
    if not isinstance(resp, str) or len(resp) < 1500:
        continue
    h = hashlib.new("sha256")
    h.update(resp.encode("utf-8"))
    with open(os.path.join("scripts-responses", h.hexdigest()), "w") as handle:
        handle.write(resp)